In [1]:
import json
import pandas as pd

In [15]:
def flatten_json(json_data):
    data = []
    for circuit, circuit_value in json_data.items(): # circuit size
        if circuit in ["5"]:
            for benchmark, benchmark_value in circuit_value.items(): # benchmark_name
                for layout, metrics in benchmark_value.items():
                    trivial_swap = metrics.get('trivial_swap', None)
                    sabre_swap = metrics.get('sabre_swap', None)
                    lookahead_swap = metrics.get('lookahead_swap', None)
                    swap_change = ((lookahead_swap - sabre_swap) / sabre_swap * 100) if sabre_swap != 0 and sabre_swap is not None and lookahead_swap is not None else None
                    
                    trivial_depth = metrics.get('trivial_depth', None)
                    sabre_depth = metrics.get('sabre_depth', None)
                    lookahead_depth = metrics.get('lookahead_depth', None)
                    depth_change = ((lookahead_depth - sabre_depth) / sabre_depth * 100) if sabre_depth != 0 and sabre_depth is not None and lookahead_depth is not None else None

                    data.append({
                        "Benchmark": benchmark,
                        "n": circuit, # circuit size
                        "g": benchmark_value['init']['size'], # total gates
                        "d": benchmark_value['init']['depth'], # circuit depth
                        "Layout": layout,
                        "swap_b": trivial_swap, # basic swap
                        "swap_s": sabre_swap, # sabre swap
                        "swap_l": lookahead_swap, # lookahead swap
                        "Swap (%)": swap_change,
                        "d_b": trivial_depth, # basic depth
                        "d_s": sabre_depth, # sabre depth
                        "d_l": lookahead_depth, # lookahead depth
                        "Depth (%)": depth_change
                    })
    return pd.DataFrame(data)


In [6]:
def open_json_from_file():
    with open("result/benchmarking_2.json") as json_file:
        return json.load(json_file)

dict_benchmark = open_json_from_file()
print(len(dict_benchmark["5"].keys()))

15


In [16]:
df = flatten_json(dict_benchmark)
print(df)

    Benchmark  n   g   d            Layout swap_b  swap_s  swap_l  Swap (%)  \
0          dj  5  36  11              init   None     NaN     NaN       NaN   
1          dj  5  36  11         full_10_2   None     0.0     0.0       NaN   
2          dj  5  36  11          full_7_3   None     3.0     0.0    -100.0   
3          dj  5  36  11         ring_10_2   None     3.0     3.0       0.0   
4          dj  5  36  11          ring_7_3   None     3.0     3.0       0.0   
..        ... ..  ..  ..               ...    ...     ...     ...       ...   
145    wstate  5  73  45          grid_9_3   None     0.0    12.0       NaN   
146    wstate  5  73  45          grid_4_5   None     0.0     9.0       NaN   
147    wstate  5  73  45          line_5_4   None     0.0    15.0       NaN   
148    wstate  5  73  45  t_horizontal_5_4   None     0.0     6.0       NaN   
149    wstate  5  73  45    t_vertical_5_4   None     0.0     6.0       NaN   

      d_b   d_s   d_l  Depth (%)  
0    None   NaN 

In [17]:
def save_latex_to_file(latex, circuit_size):
    with open(f"result/table_{circuit_size}.tex", "w") as file:
        file.write(latex)

In [18]:
df_n_5 = df[df["n"] == "5"]
latex_n_5 = df_n_5.to_latex(index=False)
save_latex_to_file(latex_n_5, 5)